In [0]:
from google.colab import files
files.upload()

{}

In [0]:
!unzip signdata.zip

In [0]:
from __future__ import absolute_import, division, print_function

# Import TensorFlow >= 1.10 and enable eager execution
import tensorflow as tf

tf.enable_eager_execution()

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import time

print(tf.__version__)

1.13.1


In [0]:
import pandas as pd
train=pd.read_csv('/content/signdata/PHOENIX-2014-T.train.corpus.csv',sep='|')
test=pd.read_csv('/content/signdata/PHOENIX-2014-T.test.corpus.csv',sep='|')
dev=pd.read_csv('/content/signdata/PHOENIX-2014-T.dev.corpus.csv',sep='|')
dev.columns

Index(['name', 'video', 'start', 'end', 'speaker', 'orth', 'translation'], dtype='object')

In [0]:
train.sample(10)

,name,video,start,end,speaker,orth,translation
2665,25August_2009_Tuesday_heute-3320,25August_2009_Tuesday_heute-3320/1/*.png,-1,-1,Signer01,JETZT WUENSCHEN ABEND MITTEILEN,und jetzt wünsche ich ihnen noch einen schönen...
6972,05July_2010_Monday_heute-8692,05July_2010_Monday_heute-8692/1/*.png,-1,-1,Signer01,JETZT REGEN BESONDERS OST REGEN,ja schauer und gewitter jetzt vor allem in der...
953,09August_2009_Sunday_tagesschau-1194,09August_2009_Sunday_tagesschau-1194/1/*.png,-1,-1,Signer01,BESONDERS NORD REGION MEHR SCHAUER,vor allem im norden und südlich der donau ein ...
3043,23February_2011_Wednesday_heute-3766,23February_2011_Wednesday_heute-3766/1/*.png,-1,-1,Signer05,WETTER HEUTE MEHR GRAU HEUTE BLAU HIMMEL WUERZ...,unser wetter wird wieder ein bisschen grauer a...
1772,14October_2010_Thursday_heute-2190,14October_2010_Thursday_heute-2190/1/*.png,-1,-1,Signer07,MORGEN IX BISSCHEN WOLKE NORD DEUTSCH WOLKE SO...,morgen wird es trübe sein in norddeutschland e...
6056,09December_2010_Thursday_heute-7559,09December_2010_Thursday_heute-7559/1/*.png,-1,-1,Signer07,ENORM GLATT GEFAHR UND SCHNEE REGEN SUEDOST WI...,es kann sehr glatt werden und auch der schneer...
5710,30March_2011_Wednesday_tagesschau-7128,30March_2011_Wednesday_tagesschau-7128/1/*.png,-1,-1,Signer05,OST REGION HEUTE NACHT WOLKE IX WAHRSCHEINLICH...,in der osthälfte zeigen sich heute nacht noch ...
6438,11November_2009_Wednesday_tagesschau-8036,11November_2009_Wednesday_tagesschau-8036/1/*.png,-1,-1,Signer03,SCHON MORGEN IM-VERLAUF SUEDWEST REGION ANDERS...,bereits im laufe des tages erreichen uns von s...
2637,09July_2011_Saturday_tagesschau-3288,09July_2011_Saturday_tagesschau-3288/1/*.png,-1,-1,Signer01,SKANDINAVIEN TIEF KOMMEN MORGEN MEISTENS STROEMEN,das tief das nach skandinavien zieht bestimmt ...
6417,02December_2011_Friday_tagesschau-8011,02December_2011_Friday_tagesschau-8011/1/*.png,-1,-1,Signer04,TAG NORDWEST KOMMEN REGEN IX KOENNEN SONNE REG...,am tag kommt im nordwesten regen auf sonst kan...


In [0]:
gertrain=train['translation'].tolist()
glosstrain=train['orth'].tolist()

gertest=test['translation'].tolist()
glosstest=test['orth'].tolist()

gerdev=dev['translation'].tolist()
glossdev=dev['orth'].tolist()

In [0]:
#data.head()

In [0]:
#len(german[1])

In [0]:
german=[]
gloss=[]
for i in range(len(gertrain)):
  german.append(gertrain[i])
  gloss.append(glosstrain[i])

for i in range(len(gertest)):
  german.append(gertest[i])
  gloss.append(glosstest[i])

for i in range(len(gerdev)):
  german.append(gerdev[i])
  gloss.append(glossdev[i])

In [0]:
len(gertrain)+len(gertest)+len(gerdev)

8257

In [0]:
len(german)

8257

In [0]:
original_word_pairs=[[gloss[i],german[i]] for i in range(len(gloss))]

data = pd.DataFrame(original_word_pairs, columns=["eng", "es"])

In [0]:
data.sample(10)

,eng,es
5196,ENORM WARM ZWANZIG BIS FUENF ZWANZIG GRAD poss...,also spätsommerlich zwanzig bis fünfundzwanzig...
5462,NACHT NULL NORDOST MINUS ZWOELF ALPEN KLAR,heute nacht null grad an der ostsee und bis mi...
6071,DANN HOEHER MITTE BERG GRAUPEL SCHNEE,und in den höheren mittelgebirgen kann es grau...
1853,BERG HOCH KOENNEN QUELL WOLKE NAH REGION KOENN...,über dem bergland können sich einzelne quellwo...
1303,TAG SUED FREUNDLICH SONNE NORD AUCH BISSCHEN W...,am tag ganz im süden freundliches wetter auch ...
3047,AUCH SACHSEN IX THUERINGEN IX AUCH TEIL IX BER...,auch hier in sachsen oder thüringen teilweise ...
7151,REGION TAGSUEBER DANN WOLKE DANN SUEDWEST GEWI...,vor allem über dem bergland können sich im tag...
5129,ABER NACHT REGION FROST neg-HABEN NORDOST EINS...,aber die nacht bleibt fast überall frostfrei d...
2089,MORGEN FUENFZEHN NEUNZEHN ZWANZIG BIS ZWEI ZWA...,morgen fünfzehn bis neunzehn entlang des rhein...
698,STURM DABEI NORD SCHWER STURM WENN BERG IX REG...,sturmböen sind mit dabei im norden auch schwer...


In [0]:
data.shape

(8257, 2)

In [0]:
# Converts the unicode file to ascii
def unicode_to_ascii(s):
    """
    Normalizes latin chars with accent to their canonical decomposition
    """
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')

def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())
    
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ." 
    # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    
    w = w.rstrip().strip()
    
    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    w = '<start> ' + w + ' <end>'
    return w

In [0]:
# Now we do the preprocessing using pandas and lambdas
data["eng"] = data.eng.apply(lambda w: preprocess_sentence(w))
data["es"] = data.es.apply(lambda w: preprocess_sentence(w))
data.sample(10)


,eng,es
8053,<start> heute nacht elf grad berg allgaeu bis ...,<start> elf grad heute nacht im allgau bis sie...
5017,<start> region west jetzt mild alpen einigerma...,<start> milder bleibt es im westen auch am alp...
1445,<start> nacht teil wolke teil klar himmel ost ...,<start> in der nacht ist es teils wolkig teils...
4499,<start> sued maessig wehen <end>,<start> im suden schwacher bis ma iger wind au...
5675,<start> heute nacht hauptsaechlich leicht fros...,<start> heute nacht verbreitet leichter frost ...
2388,<start> auch montag wolke mehr sonne wenig alp...,<start> auch am montag mehr wolken als sonne a...
5589,<start> freitag auch viel wolke mal bisschen s...,<start> am freitag ebenfalls viele wolken hier...
5157,<start> nordost wind <end>,<start> im nordosten ist es windig <end>
1015,<start> freitag wetter noch wechselhaft samsta...,<start> am freitag noch wechselhaft aber am sa...
2258,<start> nacht wolke ueberwiegend teil einige r...,<start> in der nacht liegen die wolken meist d...


In [0]:
data.columns
testgerman=data['eng'].tolist()
testgloss=data['es'].tolist()


In [0]:
len(testgerman)

8257

In [0]:
# This class creates a word -> index mapping (e.g,. "dad" -> 5) and vice-versa 
# (e.g., 5 -> "dad") for each language,
class LanguageIndex():
    def __init__(self, lang):
        """ lang are the list of phrases from each language"""
        self.lang = lang
        self.word2idx = {}
        self.idx2word = {}
        self.vocab = set()
        
        self.create_index()
        
    def create_index(self):
        for phrase in self.lang:
            # update with individual tokens
            self.vocab.update(phrase.split(' '))
            
        # sort the vocab
        self.vocab = sorted(self.vocab)

        # add a padding token with index 0
        self.word2idx['<pad>'] = 0
        
        # word to index mapping
        for index, word in enumerate(self.vocab):
            self.word2idx[word] = index + 1 # +1 because of pad token
        
        # index to word mapping
        for word, index in self.word2idx.items():
            self.idx2word[index] = word        

In [0]:
# index language using the class above
inp_lang = LanguageIndex(data["es"].values.tolist())
targ_lang = LanguageIndex(data["eng"].values.tolist())
# Vectorize the input and target languages
input_tensor = [[inp_lang.word2idx[s] for s in es.split(' ')]  for es in data["es"].values.tolist()]
target_tensor = [[targ_lang.word2idx[s] for s in eng.split(' ')]  for eng in data["eng"].values.tolist()]
input_tensor[:10]

[[2, 2523, 1734, 475, 2815, 855, 1557, 491, 442, 2976, 199, 1],
 [2,
  1471,
  1901,
  712,
  163,
  1382,
  2523,
  674,
  163,
  1436,
  2498,
  2125,
  2822,
  1548,
  1],
 [2,
  1194,
  1717,
  346,
  712,
  1105,
  1587,
  1490,
  2442,
  2190,
  1568,
  1514,
  2374,
  1321,
  1980,
  931,
  2773,
  1768,
  1514,
  350,
  2523,
  489,
  1],
 [2,
  163,
  92,
  2347,
  987,
  712,
  2579,
  2937,
  2370,
  1319,
  1978,
  1750,
  962,
  2523,
  1197,
  1469,
  1896,
  728,
  675,
  1880,
  1],
 [2, 1019, 663, 2878, 750, 2106, 2661, 66, 1194, 1717, 1],
 [2, 1194, 625, 1105, 1587, 1735, 1633, 92, 1747, 335, 2121, 1006, 1],
 [2,
  1557,
  56,
  2377,
  2821,
  1105,
  1638,
  335,
  849,
  1197,
  446,
  1396,
  335,
  2118,
  1006,
  1],
 [2,
  92,
  790,
  1254,
  712,
  1197,
  446,
  1782,
  2374,
  1382,
  2523,
  1318,
  1898,
  2681,
  1161,
  1],
 [2, 2190, 2725, 1514, 1980, 2523, 969, 475, 2541, 163, 92, 2864, 261, 1],
 [2, 92, 2381, 106, 2106, 2764, 1]]

In [0]:
target_tensor[:10]

[[2, 408, 917, 526, 182, 974, 255, 1],
 [2, 593, 629, 199, 629, 432, 813, 432, 1],
 [2, 572, 367, 536, 798, 126, 771, 432, 629, 593, 315, 165, 1],
 [2, 778, 587, 629, 315, 432, 486, 629, 859, 629, 1],
 [2, 938, 474, 738, 572, 1],
 [2, 267, 367, 536, 694, 267, 717, 327, 1],
 [2, 526, 788, 322, 919, 367, 553, 120, 286, 967, 451, 402, 714, 967, 327, 1],
 [2, 275, 594, 432, 629, 746, 629, 878, 813, 432, 1],
 [2, 630, 898, 629, 315, 53, 935, 1],
 [2, 788, 126, 322, 1]]

In [0]:
def max_length(tensor):
    return max(len(t) for t in tensor)

In [0]:
# calculate the max_length of input and output tensor
max_length_inp, max_length_tar = max_length(input_tensor), max_length(target_tensor)

In [0]:
print(max_length_inp, max_length_tar)

54 34


In [0]:
def pad_sequences(x, max_len):
    padded = np.zeros((max_len), dtype=np.int64)
    if len(x) > max_len: padded[:] = x[:max_len]
    else: padded[:len(x)] = x
    return padded

In [0]:
# inplace padding
input_tensor = [pad_sequences(x, max_length_inp) for x in input_tensor]
target_tensor = [pad_sequences(x, max_length_tar) for x in target_tensor]
len(target_tensor)

8257

In [0]:
len(gertrain)

7096

In [0]:
len(gertest)+len(gertrain)

7738

In [0]:
#len(input_tensor)
input_tensor_train=input_tensor[0:len(gertrain)]
input_tensor_val=input_tensor[len(gertest)+len(gertrain):len(gertrain)+len(gertest)+len(gerdev)]
target_tensor_train=target_tensor[0:len(gertrain)]
target_tensor_val=target_tensor[len(gertest)+len(gertrain) : len(gertrain)+len(gertest)+len(gerdev)]

In [0]:
# Creating training and validation sets using an 80-20 split
#input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val)

(7096, 7096, 519, 519)

In [0]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
N_BATCH = BUFFER_SIZE//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word2idx)
vocab_tar_size = len(targ_lang.word2idx)

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [0]:
vocab_inp_size

2978

In [0]:
vocab_tar_size

976

In [0]:
def gru(units):
  # If you have a GPU, we recommend using CuDNNGRU(provides a 3x speedup than GRU)
  # the code automatically does that.
  if tf.test.is_gpu_available():
    return tf.keras.layers.CuDNNGRU(units, 
                                    return_sequences=True, 
                                    return_state=True, 
                                    recurrent_initializer='glorot_uniform')
  else:
    return tf.keras.layers.GRU(units, 
                               return_sequences=True, 
                               return_state=True, 
                               recurrent_activation='sigmoid', 
                               recurrent_initializer='glorot_uniform')

In [0]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.enc_units)
        
    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)        
        return output, state
    
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

In [0]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.dec_units)
        self.fc = tf.keras.layers.Dense(vocab_size)
        
        # used for attention
        self.W1 = tf.keras.layers.Dense(self.dec_units)
        self.W2 = tf.keras.layers.Dense(self.dec_units)
        self.V = tf.keras.layers.Dense(1)
        
    def call(self, x, hidden, enc_output):
        # enc_output shape == (batch_size, max_length, hidden_size)
        
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(hidden, 1)
        
        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying tanh(FC(EO) + FC(H)) to self.V
        score = self.V(tf.nn.tanh(self.W1(enc_output) + self.W2(hidden_with_time_axis)))
        
        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)
        
        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * enc_output
        context_vector = tf.reduce_sum(context_vector, axis=1)
        
        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)
        
        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        
        # passing the concatenated vector to the GRU
        output, state = self.gru(x)
        
        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))
        
        # output shape == (batch_size * 1, vocab)
        x = self.fc(output)
        
        return x, state, attention_weights
        
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.dec_units))

In [0]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

In [0]:
optimizer = tf.train.AdamOptimizer()


def loss_function(real, pred):
  mask = 1 - np.equal(real, 0)
  loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred) * mask
  return tf.reduce_mean(loss_)

In [0]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [0]:
loss=[]
EPOCHS = 100

for epoch in range(EPOCHS):
    start = time.time()
    
    hidden = encoder.initialize_hidden_state()
    total_loss = 0
    
    for (batch, (inp, targ)) in enumerate(dataset):
        loss = 0
        
        with tf.GradientTape() as tape:
            enc_output, enc_hidden = encoder(inp, hidden)
            
            dec_hidden = enc_hidden
            
            dec_input = tf.expand_dims([targ_lang.word2idx['<start>']] * BATCH_SIZE, 1)       
            
            # Teacher forcing - feeding the target as the next input
            for t in range(1, targ.shape[1]):
                # passing enc_output to the decoder
                predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
                
                loss += loss_function(targ[:, t], predictions)
                
                # using teacher forcing
                dec_input = tf.expand_dims(targ[:, t], 1)
        
        batch_loss = (loss / int(targ.shape[1]))
        
        total_loss += batch_loss
        
        variables = encoder.variables + decoder.variables
        
        gradients = tape.gradient(loss, variables)
        
        optimizer.apply_gradients(zip(gradients, variables))
        
        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                         batch,
                                                         batch_loss.numpy()))
    # saving (checkpoint) the model every 2 epochs
    if (epoch + 1) % 2 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)
    
    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                        total_loss / N_BATCH))
    #loss.append([total_loss / N_BATCH])
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Instructions for updating:
Colocations handled automatically by placer.
Epoch 1 Batch 0 Loss 1.7967
Epoch 1 Batch 100 Loss 1.1467
Epoch 1 Loss 1.3189
Time taken for 1 epoch 68.45653796195984 sec

Epoch 2 Batch 0 Loss 1.1316
Epoch 2 Batch 100 Loss 1.0740
Epoch 2 Loss 1.0980
Time taken for 1 epoch 67.29557275772095 sec

Epoch 3 Batch 0 Loss 0.9729
Epoch 3 Batch 100 Loss 1.0406
Epoch 3 Loss 1.0114
Time taken for 1 epoch 66.76883697509766 sec

Epoch 4 Batch 0 Loss 0.9912
Epoch 4 Batch 100 Loss 0.8869
Epoch 4 Loss 0.9361
Time taken for 1 epoch 67.87451028823853 sec

Epoch 5 Batch 0 Loss 0.7778
Epoch 5 Batch 100 Loss 0.7511
Epoch 5 Loss 0.8582
Time taken for 1 epoch 66.4530258178711 sec

Epoch 6 Batch 0 Loss 0.7100
Epoch 6 Batch 100 Loss 0.7478
Epoch 6 Loss 0.7832
Time taken for 1 epoch 66.07375645637512 sec

Epoch 7 Batch 0 Loss 0.6770
Epoch 7 Batch 100 Loss 0.7486
Epoch 7 Loss 0.7174
Time taken for 1 epoch 67.84517979621887 sec

Epoch 8 Batch 0 Loss 0.6582
Epoch 8 Batch 100 Loss 0.5854
Epo

In [0]:
def evaluate(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    attention_plot = np.zeros((max_length_targ, max_length_inp))
    
    sentence = preprocess_sentence(sentence)

    inputs = [inp_lang.word2idx[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')
    inputs = tf.convert_to_tensor(inputs)
    
    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word2idx['<start>']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)
        
        # storing the attention weights to plot later on
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += targ_lang.idx2word[predicted_id] + ' '

        if targ_lang.idx2word[predicted_id] == '<end>':
            return result, sentence, attention_plot
        
        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention_plot

In [0]:
# function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attention, cmap='viridis')
    
    fontdict = {'fontsize': 14}
    
    ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
    ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

    plt.show()

In [0]:
def translate(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    result, sentence, attention_plot = evaluate(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)
        
    #print('Input: {}'.format(sentence))
    #print('Predicted translation: {}'.format(result))
    
    #attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
    #plot_attention(attention_plot, sentence.split(' '), result.split(' '))
    return(result)

In [0]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [0]:
te=pd.read_csv('/content/signdata/PHOENIX-2014-T.test.corpus.csv',sep='|')
ge=te['translation'].tolist()
go=te['orth'].tolist()

In [0]:
len(go)

642

In [0]:
#ger[1]

In [0]:
from nltk.translate.bleu_score import sentence_bleu

In [0]:
pred=[]
bleu=[]
no=[]

In [0]:
pred=[]
bleu1=[]
bleu2=[]
bleu3=[]
bleu4=[]
no=[]

In [0]:
import csv

In [0]:
len(train)
len(test)

642

In [0]:
from nltk.translate.bleu_score import sentence_bleu
reference = [['this', 'is', 'a', 'test'], ['this', 'is' 'test']]
candidate = ['this', 'is', 'a', 'test']
score = sentence_bleu(reference, candidate)
score

1.0

In [0]:
def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())
    
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ." 
    # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    
    w = w.rstrip().strip()
    
    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    #w =   w 
    return w

In [0]:

for i in range(len(go)):
  #no.append(i)
  pred=translate(ge[i], encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ=34)
  #pred.append(translate(go[i], encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ=20))
  # cumulative BLEU scores
  #print(pred)
  pred1=pred.split(' ')
  reference = [ pred1[0:-2] ]
  print(reference)
  candidate = (preprocess_sentence(go[i]).split(' '))
  print(candidate)
  
  bleu1=(sentence_bleu(reference, candidate, weights=(1, 0, 0, 0)))
  bleu2=(sentence_bleu(reference, candidate, weights=(0.5, 0.5, 0, 0)))
  bleu3=(sentence_bleu(reference, candidate, weights=(0.33, 0.33, 0.33, 0)))
  bleu4=(sentence_bleu(reference, candidate, weights=(0.25, 0.25, 0.25, 0.25)))
  
  bleu= sentence_bleu(reference, candidate)
  #print('Cumulative 1-gram: %f' % sentence_bleu(reference, candidate, weights=(1, 0, 0, 0)))
  #print('Cumulative 2-gram: %f' % sentence_bleu(reference, candidate, weights=(0.5, 0.5, 0, 0)))
  #print('Cumulative 3-gram: %f' % sentence_bleu(reference, candidate, weights=(0.33, 0.33, 0.33, 0)))
  #print('Cumulative 4-gram: %f' % sentence_bleu(reference, candidate, weights=(0.25, 0.25, 0.25, 0.25)))
  row=[go[i],ge[i],pred,bleu1,bleu2,bleu3,bleu4,bleu]
  with open('nmttest.csv', 'a') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerow(row)

  csvFile.close()
  print(i," is done")

[['regen', 'regen', 'schnee', 'berg', 'sued', 'region', 'schauer', 'region', 'klar', 'region']]
['regen', 'schnee', 'region', 'verschwinden', 'nord', 'regen', 'koennen', 'region', 'stern', 'koennen', 'sehen']
0  is done
[['donnerstag', 'donnerstag', 'nord', 'regen', 'nord', 'wechselhaft', 'wolke', 'wechselhaft', 'mehr', 'sonne']]
['donnerstag', 'nordwest', 'regen', 'region', 'sonne', 'wolke', 'wechselhaft', 'dann', 'freitag', 'aehnlich', 'wetter']
1  is done
[['nord', 'region', 'kommen', 'meistens', 'zwei', 'regen']]
['kraeftig', 'ab', 'morgen', 'frueh', 'meistens', 'schnee', 'schneien', 'kalt', 'regen']
2  is done
[['samstag', 'sonne']]
['wochenende', 'sonne', 'samstag', 'schoen', 'temperatur', 'bis', 'siebzehn', 'grad', 'region']
3  is done
[['deutsch', 'land', 'morgen', 'wolke', 'kommen']]
['deutsch', 'land', 'morgen', 'hoch', 'druck', 'kommen', 'wolke', 'aufloesen']
4  is done


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


[['sonntag', 'nord', 'sonne', 'wolke', 'bisschen', 'schauer', 'gewitter', 'koennen']]
['sonntag', 'naechste', 'nordwest', 'wolke', 'sonne', 'wolke', 'gewitter', 'regen', 'dabei']
5  is done
[['schauer', 'bisschen', 'gewitter', 'sued', 'hauptsaechlich']]
['wahrscheinlich', 'schauer', 'gewitter', 'stark']
6  is done
[['jetzt', 'in', 'kommend', 'wie', 'immer']]
['im', 'verlauf', 'bleiben', 'kalt', 'enorm', 'schnee', 'koennen']
7  is done
[['dann', 'wochenende', 'dann', 'wochenende', 'dann', 'wochenende', 'dann', 'bisschen', 'mehr', 'kalt']]
['wochenende', 'ix', 'mehr', 'kalt']
8  is done
[['morgen', 'auch', 'hoch', 'druck', 'kommen', 'wolke', 'himmel', 'viel', 'sonne', 'region']]
['morgen', 'dann', 'herbst', 'mischung', 'hoch', 'nebel', 'wolke', 'sonne']
9  is done
[['spaeter', 'im', 'verlauf', 'im', 'verlauf', 'im', 'verlauf', 'im', 'verlauf', 'im', 'verlauf', 'im', 'verlauf', 'im', 'verlauf', 'im', 'verlauf', 'im', 'verlauf', 'im', 'verlauf', 'im', 'verlauf', 'im', 'verlauf', 'im', 'ver

In [0]:
len(pred)

67

In [0]:
import pandas as pd
nmt=pd.read_csv('/content/nmttest.csv')
#tdata.head()

In [0]:
nmt.columns

Index(['REGEN SCHNEE REGION VERSCHWINDEN NORD REGEN KOENNEN REGION STERN KOENNEN SEHEN',
       'regen und schnee lassen an den alpen in der nacht nach im norden und nordosten fallen hier und da schauer sonst ist das klar',
       'regen regen schnee berg sued region schauer region klar region <end> ',
       '0.45454545454545453', '0.21320071635561047', '0.3605790708802297',
       '0.46173663094410267', '0.46173663094410267.1'],
      dtype='object')

In [0]:
txt=nmt[nmt.columns[0]].tolist()
pred=nmt[nmt.columns[2]].tolist()
b1=nmt[nmt.columns[3]].tolist()
b2=nmt[nmt.columns[4]].tolist()
b3=nmt[nmt.columns[5]].tolist()
b4=nmt[nmt.columns[6]].tolist()
b=nmt[nmt.columns[7]].tolist()

In [0]:
def Nmaxelements(list1, N): 
    final_list = [] 
  
    for i in range(0, N):  
        max1 = 0
          
        for j in range(len(list1)):      
            if list1[j] > max1: 
                max1 = list1[j]; 
                  
        list1.remove(max1); 
        final_list.append(max1) 
          
    print(final_list)
    return(final_list)
# Calling the function 
N=100
bs1=Nmaxelements(b1, N) 
bs2=Nmaxelements(b2, N) 
bs3=Nmaxelements(b3, N) 
bs4=Nmaxelements(b4, N) 
bs=Nmaxelements(b, N) 


[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8888888888888888, 0.8888888888888888, 0.875, 0.8571428571428571, 0.8571428571428571, 0.8571428571428571, 0.8571428571428571, 0.8571428571428571, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8187307530779819, 0.8143536762323635, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.7788007830714049, 0.7788007830714049, 0.7788007830714049, 0.7788007830714049, 0.7777777777777778, 0.7777777777777778, 0.7777777777777778, 0.7777777777777778, 0.7777777777777778, 0.7777777777777778, 0.7777777777777778, 0.7721847897615209, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.7430381997858699, 0.7165313105737893, 0.7165313105737893, 0.7165313105737893, 0.7165313105737893, 0.7165313105737893, 0.7142857142857143, 0.7142857142857143, 0.7142857142857143, 0.7054014374088451, 0.7054014374088451, 0.7, 0.

In [0]:
txt=nmt[nmt.columns[0]].tolist()
pred=nmt[nmt.columns[2]].tolist()
b1=nmt[nmt.columns[3]].tolist()
b2=nmt[nmt.columns[4]].tolist()
b3=nmt[nmt.columns[5]].tolist()
b4=nmt[nmt.columns[6]].tolist()
b=nmt[nmt.columns[7]].tolist()

In [0]:
for i in range(len(b4)):
  if(b4[i]==1):
    print('txt:  ',txt[i])
    print('pre:  ',pred[i])
    print("i",i)


txt:   GUT ABEND LIEB ZUSCHAUER
pre:   gut abend lieb zuschauer <end> 
i 325
txt:   MORGEN SCHWACH MAESSIG WEHEN
pre:   morgen schwach maessig wehen <end> 
i 370
txt:   NORD NUR SECHSZEHN GRAD
pre:   nord nur sechszehn grad <end> 
i 424
txt:   JETZT WETTER MORGEN DONNERSTAG SIEBZEHN DEZEMBER
pre:   jetzt wetter morgen donnerstag siebzehn dezember <end> 
i 431
txt:   SONST REGION FREUNDLICH
pre:   sonst region freundlich <end> 
i 551
txt:   BLEIBEN WIND
pre:   bleiben wind <end> 
i 563
txt:   JETZT WETTER MORGEN SAMSTAG ZWOELF SEPTEMBER
pre:   jetzt wetter morgen samstag zwoelf september <end> 
i 633


In [0]:
b1=nmt[nmt.columns[3]].tolist()
b2=nmt[nmt.columns[4]].tolist()
b3=nmt[nmt.columns[5]].tolist()
b4=nmt[nmt.columns[6]].tolist()
b=nmt[nmt.columns[7]].tolist()

In [0]:
#position in nmt
bd1=[]
bd2=[]
bd3=[]
bd4=[]
bd=[]
for i in range(len(bs1)):
  #print(bs1[i])
  bd1.append(b1.index(bs1[i]))
  bd2.append(b2.index(bs2[i]))
  bd3.append(b3.index(bs3[i]))
  bd4.append(b4.index(bs4[i]))
  bd.append(b4.index(bs[i]))

In [0]:
bl1 = list(set(bd1))
bl2 = list(set(bd2))
bl3 = list(set(bd3))
bl4 = list(set(bd4))
bl=list(set(bd))

In [0]:
bl

[144,
 18,
 20,
 152,
 27,
 169,
 426,
 428,
 184,
 189,
 193,
 449,
 67,
 325,
 77,
 344,
 345,
 95,
 356,
 231,
 364,
 118]

In [0]:
for i in bl:
  print('original:',txt[i])
  print("predicted:",pred[i])
  print("___________________________________________________")
print("=====================================================================")
print("=====================================================================")

print("                                                                                   ")
print("                                                                                   ")
print("                                                                                   ")
print("                                                                                   ")
print("                                                                                   ")

for i in bl1:
  print('original:',txt[i])
  print("predicted:",pred[i])
  print("___________________________________________________")
print("=====================================================================")
print("=====================================================================")

print("                                                                                   ")
print("                                                                                   ")
print("                                                                                   ")
print("                                                                                   ")
print("                                                                                   ")

for i in bl2:
  print('original:',txt[i])
  print("predicted:",pred[i])
  print("___________________________________________________")
print("=====================================================================")
print("=====================================================================")

print("                                                                                   ")
print("                                                                                   ")
print("                                                                                   ")
print("                                                                                   ")
print("                                                                                   ")

for i in bl3:
  print('original:',txt[i])
  print("predicted:",pred[i])
  print("___________________________________________________")
print("=====================================================================")

print("                                                                                   ")
print("                                                                                   ")
print("                                                                                   ")
print("                                                                                   ")
print("                                                                                   ")

print("=====================================================================")

for i in bl4:
  print('original:',txt[i])
  print("predicted:",pred[i])
  print("___________________________________________________")
print("=====================================================================")
print("                                                                                   ")
print("                                                                                   ")
print("                                                                                   ")
print("                                                                                   ")
print("                                                                                   ")
print("=====================================================================")



original: und nun die wettervorhersage für morgen samstag den einunddreißigsten oktober
predicted: jetzt wetter morgen samstag eins zwanzig oktober <end> 
___________________________________________________
original: heute nacht ist es im süden teils klar teilweise aber auch nebel
predicted: heute nacht sued klar teil auch nebel <end> 
___________________________________________________
original: morgen scheint dann wieder überall die sonne
predicted: morgen region sonne <end> 
___________________________________________________
original: am samstag viel sonnenschein vor allem an alpen und küsten ist es auch mal wolkig
predicted: samstag viel sonne besonders alpen kueste wolke enorm wolke <end> 
___________________________________________________
original: am dienstag ist es teils freundlich teils stärker bewölkt einige regenschauer sind dabei mitunter auch gewitter
predicted: dienstag teilweise freundlich teil wolke bisschen regen teilweise gewitter <end> 
____________________________

In [0]:
!pip install googletrans

  Stored in directory: /root/.cache/pip/wheels/50/d6/e7/a8efd5f2427d5eb258070048718fa56ee5ac57fd6f53505f95
Successfully built googletrans


In [0]:
from googletrans import Translator
translator = Translator()


In [0]:
for i in bl1:
  print(txt[i])

dort morgen sieben grad nordöstlich der elbe teilweise nur minus ein grad
ganz ähnliche temperaturen wie heute zwischen sechs und elf grad
am tag in der nordhälfte etwas regen schnee oder gefrierender regen
das tief sorgt auch noch für wolken aber es fällt nur wenig regen
heute nacht ist es im süden teils klar teilweise aber auch nebel
temperaturen nur so um die null grad am kältesten im norden bis minus fünf grad
am samstag meist freundlich und überwiegend trocken
teilweise ist es auch klar
heute nacht schwül warme dreizehn bis achtzehn grad
dort morgen bis zweiundzwanzig grad
sonst ist es recht freundlich
in der nacht sinken die temperaturen auf vierzehn bis sieben grad
samstag fällt auch noch schnee
montag und dienstag wechselhaft hier und da zeigt sich aber auch die sonne
am samstag überwiegend freundlich meist trocken und wärmer
guten abend liebe zuschauer
unter wolken heute nacht frostfreie zwei bis acht grad
am niederrhein heute nacht vierzehn im osten örtlich nur sechs grad
lie

In [0]:
txt=nmt[nmt.columns[1]].tolist()
pred=nmt[nmt.columns[2]].tolist()

In [0]:
translator.translate('안녕하세요.')

In [0]:
btt1=translator.translate([txt[i] for i in bl1], dest='en')
bpt1=translator.translate([pred[i] for i in bl1], dest='en')


btt2=translator.translate([txt[i] for i in bl2], dest='en')
bpt2=translator.translate([pred[i] for i in bl2], dest='en')

btt3=translator.translate([txt[i] for i in bl3], dest='en')
bpt3=translator.translate([pred[i] for i in bl3], dest='en')


btt4=translator.translate([txt[i] for i in bl4], dest='en')
bpt4=translator.translate([pred[i] for i in bl4], dest='en')

btt=translator.translate([txt[i] for i in bl], dest='en')
bpt=translator.translate([pred[i] for i in bl], dest='en')

In [0]:
for translation,translations in zip(btt1,bpt1):
  #for  in bpt1:
  print("origin:",translation.origin)
  print("pred  :",translations.origin)
  print("origin:",translation.text)
  print("pred  :",translations.text)
  print(" ____________________________________________________________________________________________ ")

origin: dort morgen sieben grad nordöstlich der elbe teilweise nur minus ein grad
pred  : morgen ix nord region sieben grad region nur minus zwei grad <end> 
origin: there tomorrow seven degrees northeast of the Elbe sometimes only minus one degree
pred  : tomorrow ix north region seven degrees region only minus two degrees <end>
 ____________________________________________________________________________________________ 
origin: ganz ähnliche temperaturen wie heute zwischen sechs und elf grad
pred  : temperatur gleich heute nacht sechs bis elf <end> 
origin: very similar temperatures as today between six and eleven degrees
pred  : temperature equal tonight six to eleven <end>
 ____________________________________________________________________________________________ 
origin: am tag in der nordhälfte etwas regen schnee oder gefrierender regen
pred  : tag nord regen schnee regen glatt <end> 
origin: During the day in the northern half, some rain or freezing rain
pred  : day north rai

In [0]:
for translation,translations in zip(btt2,bpt2):
  #for  in bpt1:
  print("origin:",translation.origin)
  print("pred  :",translations.origin)
  print("origin:",translation.text)
  print("pred  :",translations.text)
  print(" ____________________________________________________________________________________________ ")

origin: am donnerstag regen in der nordhälfte in der südhälfte mal sonne mal wolken ähnliches wetter dann auch am freitag
pred  : donnerstag nord regen sued wechselhaft sonne wolke wechselhaft auch freitag <end> 
origin: on thursday rain in the north half in the southern half times sun like clouds similar weather then also on friday
pred  : thursday north rain sued changeable sun cloud changeable also friday <end>
 ____________________________________________________________________________________________ 
origin: deutschland liegt morgen unter hochdruckeinfluss der die wolken weitgehend vertreibt
pred  : deutschland morgen region morgen tief kommen morgen <end> 
origin: tomorrow, germany will be under the influence of high pressure, which will largely drive out the clouds
pred  : germany morning region tomorrow come tomorrow morning <end>
 ____________________________________________________________________________________________ 
origin: und zum wochenende wird es dann sogar wieder

In [0]:
for translation,translations in zip(btt3,bpt3):
  #for  in bpt1:
  print("origin:",translation.origin)
  print("pred  :",translations.origin)
  print("origin:",translation.text)
  print("pred  :",translations.text)
  print(" ____________________________________________________________________________________________ ")

In [0]:
for translation,translations in zip(btt4,bpt4):
  #for  in bpt1:
  print("origin:",preprocess_sentence(translation.origin))
  print("pred  :",translations.origin)
  print("origin:",translation.text)
  print("pred  :",translations.text)
  print(" ____________________________________________________________________________________________ ")

In [0]:
import pandas as pd
fin=pd.read_csv('./nmttest.csv')

In [0]:
fin.head()

,REGEN SCHNEE REGION VERSCHWINDEN NORD REGEN KOENNEN REGION STERN KOENNEN SEHEN,regen und schnee lassen an den alpen in der nacht nach im norden und nordosten fallen hier und da schauer sonst ist das klar,regen regen schnee berg sued region schauer region klar region <end>,0.45454545454545453,0.21320071635561047,0.3605790708802297,0.46173663094410267,0.46173663094410267.1
0,DONNERSTAG NORDWEST REGEN REGION SONNE WOLKE W...,am donnerstag regen in der nordhälfte in der s...,donnerstag donnerstag nord regen nord wechselh...,0.454545,0.213201,0.360579,0.461737,0.461737
1,KRAEFTIG AB MORGEN FRUEH MEISTENS SCHNEE SCHNE...,vom nordmeer zieht ein kräftiges tief heran un...,nord region kommen meistens zwei regen <end>,0.222222,0.471405,0.608751,0.686589,0.686589
2,WOCHENENDE SONNE SAMSTAG SCHOEN TEMPERATUR BIS...,sonnig geht es auch ins wochenende samstag ein...,samstag sonne <end>,0.222222,0.471405,0.608751,0.686589,0.686589
3,DEUTSCH LAND MORGEN HOCH DRUCK KOMMEN WOLKE AU...,deutschland liegt morgen unter hochdruckeinflu...,deutsch land morgen wolke kommen <end>,0.625000,0.422577,0.313551,0.415351,0.415351
4,SONNTAG NAECHSTE NORDWEST WOLKE SONNE WOLKE GE...,am sonntag im nordwesten eine mischung aus son...,sonntag nord sonne wolke bisschen schauer gewi...,0.444444,0.235702,0.385265,0.485492,0.485492


In [0]:
number4=fin[]